# Script for Running MK Modelling from Library

## Imports

In [ ]:
import sys
sys.path.append("../py_src")

from glob import glob
import os

import numpy as np
import matplotlib.pyplot as plt

from mk_modelling import MkModeller

from string import ascii_uppercase # for labeling

## Defining k values and Constants

In [ ]:
k_vals = np.array([
    [1.95396E+13, 1.41811E-14], # k_1b, k_1f
    [1046412597, 116.0938098], # k_2b, k_2f
    [602994700.8, 32.39167923],
    [8022.839446, 10384632.47],
    [1674.78108, 4477207.393],
    [717645834.4, 1.95396E+13],
    [68237499808, 2506214.873],
    [3.52845e-7, 1.95396E+13] # k_8b, k_8f
])

constants = np.array([
    1., # A
    np.nan, # B
    np.nan,
    np.nan,
    np.nan,
    np.nan,
    0.1,
    np.nan,
    0.1     # I
])

cap_letters = list(ascii_uppercase)[:9]

k_cap = 1.96e13
assert np.sum(k_vals > k_cap) == 0, "ONE OF YOUR K VALUES IS TOO LARGE RHYS."
# k_vals[k_vals > 1.96e13] = k_cap

## Defining timesteps and MK Modeller

In [ ]:
n_timesteps = 200
mk_modeller = MkModeller()

mk_modeller.k[...] = k_vals
mk_modeller.y_mask[:] = constants

## Running and Plotting

In [ ]:
sol = mk_modeller.solve(n_timesteps, max_step=1e-13, rtol=1e-9)

final_values = sol[-1, :]
der = sol[1:, :] - sol[:-1, :]

plt.title("Site Behaviour over Time")
plt.plot(sol)
plt.ylabel("Sitecount")
plt.xlabel("timestep")
plt.legend(cap_letters)

plt.show()


plt.title("Site Derivative over Time")
plt.plot(der)
plt.ylabel("Sitecount")
plt.xlabel("timestep")
ders = [r"$\frac{d%s}{dt}$"%letter for letter in cap_letters]
plt.legend(ders)

plt.show()

In [ ]:
print('Final Values')
for ii_let, letter in enumerate(cap_letters):
    print(letter+": %e"%final_values[ii_let])

## Run Through Excel File

In [ ]:
import openpyxl
import sqlalchemy

excel_path = os.path.expanduser('~/Downloads/pdcor.xlsx')

wb = openpyxl.load_workbook(filename=excel_path, data_only=True)

temperatures = [400, 600, 800]

start_cell = np.asarray([33, 2], dtype=np.int32)

In [ ]:
for sheet in wb.worksheets:
    print("Sheet: "+sheet.title)
    for ii_temp, temp in enumerate(temperatures):
        base_cell = start_cell + np.array([0, 4*ii_temp], dtype=np.int32)
        for ii_k in range(8):
            mk_modeller.k[ii_k, 0] = sheet.cell(
                row=base_cell[0]+ii_k,
                column=base_cell[1]+1
            ).value
            mk_modeller.k[ii_k, 1] = sheet.cell(
                row=base_cell[0]+ii_k,
                column=base_cell[1]
            ).value

        sol = mk_modeller.solve(n_timesteps, max_step=1e-13, rtol=1e-9)

        final_values = sol[-1, :]
        der = sol[1:, :] - sol[:-1, :]
        
        print("Result for T = %u"%temp)
        for ii_let, letter in enumerate(cap_letters):
            print(letter+": %e"%final_values[ii_let])
        print("RES: %e"%(
            final_values[7] * mk_modeller.k[7, 1]
        ) + "\n")

In [ ]:
wb.close()